**Notebook description:**

In [ ]:
import pandas as pd

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt 

import seaborn as sns
sns.set(style="white") #white background style for seaborn plots
sns.set(style="whitegrid", color_codes=True)


import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split 
from sklearn import metrics  
import pandas as pd

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#drive.mount('/content/drive')


In [ ]:
data = pd.read_csv('ks-projects-201801.csv')

In [ ]:
data.head(10)

Perform some data cleaning and feature engineering. Create one-hot-encoding for  
string labels.

In [ ]:
# number of days between launch data and deadline
def duration_of_project():
    duration = []
    
    for launch, deadline in zip(data['launched'], data['deadline']):
        
        launch = (launch.split())[0]
        launchT = [x.strip() for x in launch.split('-')]
        deadlineT = [x.strip() for x in deadline.split('-')]
    
        year_diff = (int(deadlineT[0]) - int(launchT[0]))*365
        month_diff = (int(deadlineT[1]) - int(launchT[1]))*30
        day_diff = int(deadlineT[2]) - int(launchT[2])
    
        days_offset = year_diff + month_diff + day_diff
        duration.append(days_offset)
        
    data['duration'] = duration

In [ ]:
# add column for the duration of the project
duration_of_project()
data.head(10)

In [ ]:
data.info()

In [ ]:
#getting unique categories in a list
cat = []
print("hrllo)")
for i in data['category']:
    if len(cat) == 0:
        cat.append(i)
    k = 0
    for j in cat:
        if j == i:
            k = 1
            break
    if k==0:
        cat.append(i)
#creating the unique categories successful mean
BackerMean = [0] * 160
counter = 0
for i in cat:
    count = 0
    total = 0
    for j in data['category']:
        if j==cat[counter]:
            if data['state'][count] == 'successful':
                BackerMean[counter] += data['backers'][count]
                total = total + 1
        count = count + 1
    if BackerMean[counter]!= 0:
        BackerMean[counter] = BackerMean[counter]/total
    BackerMean[counter] = round(BackerMean[counter])
    counter = counter + 1

In [ ]:
#Creates a Column stating a binary state on whether or not the project had reached the mean
#of backers in its prospective category
BackerBinary = [0] * 378661
CategoryMean = [0] * 378661
counter = 0
for i in cat:
    count = 0
    for j in data['category']:
        if j == i:
            BackerBinary[count] = data['backers'][count] - BackerMean[counter]
            CategoryMean[count] = BackerMean[counter]
            if BackerBinary[count] < 0:
                BackerBinary[count] = -1
            else:
                BackerBinary[count] = 1
        count = count+1
    counter = counter + 1

In [ ]:
data['CategoryBackerMean'] = CategoryMean
data['BackerBinary'] = BackerBinary
data.head(100)

In [ ]:
# convert state to -1, 1 for fail, succeed
data['state'].replace(['canceled'], ['failed'], inplace=True)
data['state'].replace(['failed', 'successful'], [-1, 1], inplace=True)
data.head(1)

In [ ]:
data.info();

In [ ]:
data.isnull().sum()
#how do we deal with null values

In [ ]:
#distribution of categories
print(data['category'].value_counts());
#possible new column (popular category vs unpopular category);

In [ ]:
data['state_binary'] = data['state']

In [ ]:
data.head(1)

In [ ]:
#translate state to 0 and 1
data['state_binary'].replace([-1, 1], [0, 1], inplace=True)
data.head(1000)

In [ ]:
cat = []
print("hrllo)")
for i in data['category']:
    if len(cat) == 0:
        cat.append(i)
    k = 0
    for j in cat:
        if j == i:
            k = 1
            break
    if k==0:
        cat.append(i)

BackerMean = [0] * 160
counter = 0
for i in cat:
    count = 0
    total = 0
    for j in data['category']:
        if j==cat[counter] & int(data['state'][count]) == 1:
            BackerMean[counter] += data['backers'][count]
            total = total + 1
        count = count + 1
    BackerMean[counter] = BackerMean[counter]/total
    print(BackerMean[counter], " ",counter)
    counter = counter + 1
print(BackerMean[0])

In [ ]:
print(data['main_category'].value_counts());

In [ ]:
print(data['country'].value_counts());

In [ ]:
#boolean for each category. This modified table is stored in "training"
#training=pd.get_dummies(data, columns=["main_category"])
#training.head(1)

In [ ]:
#apperently there are more possible states
sns.barplot('state_binary','duration', data=data, color="darkturquoise")
plt.show()
#Interesting to see is that the duration of unsuccessfull projects is higher. Maybe because successful projects 
#end before their initial end date. The data we got might talk about the actual end and not the initial end date

In [ ]:
print(data['state'].value_counts());

SLIM DATA IS THE TABLE THAT I USE FROM THIS POINT ON

In [ ]:
#slim data drops "undefined", "live" and "suspended"
slim_data = data.copy()
indexNames = slim_data[ (slim_data['state_binary'] != 1) & (slim_data['state_binary'] != 0) ].index
slim_data.drop(indexNames , inplace=True)
print(slim_data['state_binary'].value_counts());

In [ ]:
#Average success rate based on category
sns.barplot('state_binary','main_category', data=slim_data, color="darkturquoise")
plt.show()

In [ ]:
sns.barplot('state_binary','category', data=slim_data, color="darkturquoise")
plt.show()

In [ ]:
sns.barplot('state_binary','country', data=slim_data, color="darkturquoise")
plt.show()

In [ ]:
plt.figure(figsize=(30,10))
sns.barplot('duration', 'state_binary',data=slim_data, color="darkturquoise")
plt.show()

In [ ]:
data['duration'].describe()

Reducing Variance for the goal column

In [ ]:
#All goals below 1000 are updated to 1000
#slim_data.loc[slim_data['goal'] < 1000] = 1000
#slim_data.loc[slim_data['goal'] > 1000000] = 1000000
#slim_data.head(1000)
#slim_data['goal'].describe()

In [ ]:
# Goal histogram.
plt.figure(figsize=(30,10))
sns.distplot(np.log(data['goal']))
#plt.yticks(fig.get_yticks(), fig.get_yticks() * 100000)
#plt.ylabel('Distribution [%]', fontsize=16)
#plt.xticks(fig.get_xticks(), fig.get_xticks() * 1)


In [ ]:
#Arr = []
#for i in data['currency']:
#    exists = 0
#    if len(Arr) == 0:
#        Arr.append(i)
#    else:
#        for j in Arr:
#            if j == i:
#                exists = 1
#        if exists == 0:
#            Arr.append(i)

**Create one-hot-encodings for columns: category, main-category, currency, country**

In [ ]:
#print(slim_data['currency'][0])

unique = []
for curr in slim_data['currency']:
    if curr not in unique:
        print(curr)
        unique.append(curr)

In [ ]:
slim_data.head()

In [ ]:
# find length of name
name_length = []
for name in slim_data['name']:
    try:
        name_length.append(len(name))
    except:
        name_length.append(0)
slim_data['name_len'] = name_length

In [ ]:
# scale down data columns
sc = MinMaxScaler()

slim_data['fit_goal'] = sc.fit_transform(slim_data[['goal']])
slim_data['fit_name_len'] = sc.fit_transform(slim_data[['name_len']])
slim_data['fit_duration'] = sc.fit_transform(slim_data[['duration']])

In [ ]:
slim_data.head()

In [ ]:
training=pd.get_dummies(slim_data, columns=["main_category"])
training=pd.get_dummies(training, columns=["category"])
training=pd.get_dummies(training, columns=["country"])
training=pd.get_dummies(training, columns=["currency"])

y = training['state_binary']
X = training.loc[:, 'fit_goal':]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

X.head()

In [ ]:
#from sklearn.linear_model import LogisticRegression
#logisticRegr = LogisticRegression()
#logisticRegr.fit(X_train, y_train)
#predictions = logisticRegr.predict(X_test)
#score = logisticRegr.score(X_test, y_test)
#print(score)

In [ ]:
# build DNN

model = Sequential()

# input layer
model.add(Dense(200, activation='relu', input_dim = 214))
          
# second layer
model.add(Dense(100, activation='relu'))
          
# third layer
model.add(Dense(100, activation='relu'))
          
# fourth layer
model.add(Dense(150, activation='relu'))

# fith layer
model.add(Dense(100, activation='relu'))
          
# output layer
model.add(Dense(1, activation='sigmoid'))

# compile the model using rmsprop opt and mse loss
model.compile(optimizer = 'adam',  
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
history = model.fit(X_train, y_train, 
                    validation_split = 0.33, 
                    epochs = 50,
                    batch_size = 35)

# Evaluate the model's performance
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Training set accuracy:', train_acc)
print('Test set accuracy:', test_acc)

In [ ]:
# The history of our accuracy during training.
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# The history of our cross-entropy loss during training.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Number of epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()